In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from gensim.parsing.preprocessing import remove_stopwords 
import json


import sys
sys.path.append("../AccommodationRecommender-Airbnb/")

from utils.text_similarity import get_embedings,get_similarity

import warnings

warnings.filterwarnings("ignore")


In [8]:
# data frame

In [10]:
df = pd.read_excel("data/content_data/content_data.xlsx")


In [12]:
df.describe()

,ItemReview,ItemRatings,NumberOfGuest,NumberOfBedrooms,NumberOfBeds,NumberOfBaths,CurrentPrice,PriceBefore,PriceReview,PriceRatings
count,6148.000000,6148.000000,7175.000000,7357.0,7356.0,7221.000000,8659.000000,8659.000000,6147.000000,6147.000000
mean,4.818229,56.172739,4.864111,1.0,2.0,1.623321,132.189745,134.152904,4.818199,56.168375
std,0.226102,82.670231,2.772443,0.0,0.0,1.539842,148.884059,151.144781,0.226109,82.676248
min,2.670000,3.000000,1.000000,1.0,2.0,0.000000,7.000000,7.000000,2.670000,3.000000
25%,4.757500,10.000000,3.000000,1.0,2.0,1.000000,43.000000,44.000000,4.755000,10.000000
50%,4.880000,26.000000,4.000000,1.0,2.0,1.000000,80.000000,80.000000,4.880000,26.000000
75%,4.970000,68.000000,6.000000,1.0,2.0,2.000000,159.000000,160.000000,4.970000,68.000000
max,5.000000,873.000000,15.000000,1.0,2.0,38.000000,999.000000,999.000000,5.000000,873.000000


In [13]:
# drop duplicates
df_not_na_price  = df.dropna(subset=['ItemReview','Location','NumberOfGuest','NumberOfBedrooms', 'NumberOfBeds', 'NumberOfBaths',
                 'CurrentPrice'])

df_full_not_na = df.dropna()

In [14]:
# location in one list to get all location input
df_not_na_price[['lower_country','lower_location','lower_city','lower_state']] = df_not_na_price[['Country','Location','City','State']].apply(lambda x: x.str.lower())
df_not_na_price['location_list'] = df_not_na_price['lower_location'].apply(lambda x: list(map(str.strip,x.split(","))))


df_full_not_na[['lower_country','lower_location','lower_city','lower_state']] = df_full_not_na[['Country','Location','City','State']].apply(lambda x: x.str.lower())
df_full_not_na['location_list'] = df_full_not_na['lower_location'].apply(lambda x: list(map(str.strip,x.split(","))))

    

In [15]:
# save data sets
df_not_na_price.to_csv("data/content_data/not_na_data_price.csv",index=False)
df_full_not_na.to_csv("data/content_data/full_not_na_data.csv",index=False)

In [16]:
# text embedings for similarity
df_embedings = df_full_not_na[['ID','ItemDescription', 'Amenities', 'Comments']]

In [18]:
df_embedings[['ItemDescriptionEmbedings', 'AmenitiesEmbedings', 'CommentsEmbedings']] = \
                df_embedings[['ItemDescription', 'Amenities','Comments']]\
                .apply(lambda x: [get_embedings(text,512,True) for text in x])


In [19]:
emb_dict = df_embedings[['ID','CommentsEmbedings','ItemDescriptionEmbedings',"AmenitiesEmbedings"]].to_dict()

In [20]:
for key in emb_dict:
    if key == "ID":
        continue
        
    for inner_key,value in emb_dict[key].items():
        emb_dict[key][inner_key] = value.tolist()


In [21]:
# save embedings
with open("data/content_data/emdebings.json","w") as file:
    json.dump(emb_dict,file)